In [1]:
from dotenv import load_dotenv
import os
import msal
import requests
import json

NUMBER_INDEX = 1

env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path=env_path)

USERS_TABLE_URL = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/{os.environ.get('USERS_FILE_ID')}/workbook/worksheets/Users/tables/UsersTable/rows"
LOOKUP_TABLE_URL = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/{os.environ.get('USERS_FILE_ID')}/workbook/worksheets/Lookup/tables/LookupTable/rows"

# Enter details of AAD app registration

config = {
    'client_id': os.environ.get('CLIENT_ID'),
    'client_secret': os.environ.get('CLIENT_SECRET'),
    'authority': os.environ.get('AUTHORITY'),
    'scope': [os.environ.get('SCOPE')],
    'site_id': os.environ.get('SITE_ID'),
}


def get_users_info( msal_instance, scope):
    '''Returns a 2D list containing the user details within the inner array'''

    # First, try to lookup an access token in cache
    token_result = msal_instance.acquire_token_silent(scope, account=None)

    # If the token is available in cache, save it to a variable
    if token_result:
        print('Access token was loaded from cache')

    # If the token is not available in cache, acquire a new one from Azure AD and save it to a variable
    if not token_result:
        print(scope)
        token_result = msal_instance.acquire_token_for_client(scopes=scope)
        print(token_result)
        access_token = 'Bearer ' + token_result['access_token']
        print(f'New access token {access_token} was acquired from Azure AD')


    # Copy access_toek and specify the MS Graph API endpoint you want to call, e.g. '
    headers = {
        'Authorization': access_token
    }

    # Make a GET request to the provided url, passing the access token in a header
    users_request = requests.get(url=USERS_TABLE_URL, headers=headers)
    lookups_request = requests.get(url=LOOKUP_TABLE_URL, headers=headers)

    user_arrs = [tuple(info) for object_info in users_request.json()['value'] for info in object_info['values']]
    lookups_arrs = [tuple(info) for object_info in lookups_request.json()['value'] for info in object_info['values']]

    print(lookups_arrs)



    # user_info = [info for object_info in graph_result.json()['value'] for info in object_info['values'] if int(info[NUMBER_INDEX]) == from_number]

    print(f"user info: {user_arrs}")

    return (user_arrs, lookups_arrs) # info is already a list so user_info is a 2D list


# create an MSAL instance providing the client_id, authority and client_credential params
def get_msal_instance():
    return msal.ConfidentialClientApplication(config['client_id'], authority=config['authority'], client_credential=config['client_secret'])

msal_instance = get_msal_instance()
tables = get_users_info(msal_instance, config['scope'])

['https://graph.microsoft.com/.default']
{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6IjgxUVFhd1BHak9LaGQxTVpEUEl6Y0lOUW0wQnRPck5QV2VNRXhIUnIxUlUiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9jODE5ZjQyZC02ODhmLTRjODUtYmJjNi0xY2EwNWZkMjMzMGEvIiwiaWF0IjoxNjk3MTI3NDE1LCJuYmYiOjE2OTcxMjc0MTUsImV4cCI6MTY5NzEzMTMxNSwiYWlvIjoiRTJGZ1lMRDBEM3I3UnJQcXg2WGJBcSt1aGMzcUJRQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJRdWVyeSBNUyBHcmFwaCIsImFwcGlkIjoiNmRlMzk4ZDQtODM5NC00MTI0LWIxYmUtNWFhYmUxNGE3ZDFmIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvYzgxOWY0MmQtNjg4Zi00Yzg1LWJiYzYtMWNhMDVmZDIzMzBhLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiZGYzOTEyYWQtZTNhZi00MWZhLTk4ZjYtMGViODdhMzZiYzYwIiwicmgiOiIwLkFYSUFMZlFaeUk5b2hVeTd4aHlnWDlJekNnTUFBQUFBQUFBQXdBQUFBQUFBQUFCeUFBQS4iLCJyb2xlcyI6WyJTaXRl

In [2]:
import numpy as np
import pandas as pd
users = pd.DataFrame(data=tables[0], columns=["name", "number", "email"])

def df_replace_spaces(df):
    df.replace('', np.nan, inplace=True)
    df = df.dropna(how="all", inplace=True)
    return df

df_replace_spaces(users)
users['number'] = users["number"].astype(int)


users

,name,number,email
0,Rachmiel Teo Ren Xiang,88584969,rach@go.edu.sg
1,Shawn Tan Minyi,91681054,shawn@go.edu.sg
2,Tymothy LimJie,84885787,tymothy@go.edu.sg
3,Lim Zong Han,88252235,zonghan@go.edu.sg
4,Hariz,34567890,hariz@test.com
5,Vicky,34736482,vicky@grace.com


In [3]:
lookups = pd.DataFrame(data = tables[1], columns=["name", "reporting_officer_name", "hod_name"])

df_replace_spaces(lookups)

lookups

,name,reporting_officer_name,hod_name
0,Rachmiel Teo Ren Xiang,Lim Zong Han,Lim Zong Han
2,Tymothy LimJie,Shawn Tan Minyi,Lim Zong Han
3,Hariz,Tymothy LimJie,Shawn Tan Minyi
4,Vicky,Rachmiel Teo Ren Xiang,Shawn Tan Minyi


In [4]:
from flask import Flask, request, jsonify
import sqlite3

from datetime import datetime
from sqlalchemy.orm import Mapped, mapped_column
import os
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import event
from typing import List

basedir = os.path.join(os.getcwd())

class Config:

    # Database
    if not os.getenv("DATABASE_URL") is None:
        SQLALCHEMY_DATABASE_URI = os.getenv("DATABASE_URL")
        STATIC_FOLDER = os.path.join(os.getenv("APP_FOLDER"), "project", "static")
        UPLOADS_FOLDER = os.path.join(os.getenv("APP_FOLDER"), "project", "uploads")
    else:
        SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'chatbot.db')
        SQLALCHEMY_TRACK_MODIFICATIONS = False

app = Flask(__name__)
db = SQLAlchemy()
app.config.from_object(Config)
db.init_app(app)

class User(db.Model):

    __tablename__ = "user"
    name: Mapped[str] = mapped_column(db.String(80), primary_key=True, nullable=False)
    number: Mapped[int] = mapped_column(db.Integer(), unique=True, nullable=False)
    messages = db.relationship('Message', backref=db.backref('user'), post_update=True)

    email: Mapped[str] = mapped_column(db.String(120), unique=True, nullable=False)

    # Self-referential relationships
    reporting_officer_name: Mapped[str] = mapped_column(db.String(80), db.ForeignKey('user.name', ondelete="SET NULL"), nullable=True)
    reporting_officer = db.relationship('User', backref=db.backref('subordinates'), remote_side=[name], post_update=True, foreign_keys=[reporting_officer_name])
    
    hod_name: Mapped[str] = mapped_column(db.String(80), db.ForeignKey('user.name', ondelete="SET NULL"), nullable=True)
    hod = db.relationship('User', backref=db.backref('dept_members'), remote_side=[name], post_update=True, foreign_keys=[hod_name])

    def __init__(self, name, number, email, reporting_officer=None, hod=None):
        self.name = name
        self.number = number
        self.email = email
        self.reporting_officer = reporting_officer
        self.hod = hod


class Message(db.Model):

    __tablename__ = "message"
    id: Mapped[str] = mapped_column(db.String(50), primary_key=True, nullable=False)
    name: Mapped[int] = mapped_column(db.Integer(), db.ForeignKey('user.name', ondelete="CASCADE"), nullable=False)
    type: Mapped[str]
    body: Mapped[str] = mapped_column(db.String(20), nullable=False)
    intent: Mapped[str] = mapped_column(db.String(50), nullable=False)
    timestamp: Mapped[datetime] = mapped_column(db.DateTime, nullable=False)

    __mapper_args__ = {
        "polymorphic_identity": "message",
        "polymorphic_on": "type",
    }

    def __init__(self, id, name, body, intent, timestamp):
        self.id = id
        self.name = name
        self.body = body
        self.intent = intent
        self.timestamp = timestamp


class McDetails(Message):

    __tablename__ = "mc_details"
    id: Mapped[int] = mapped_column(db.ForeignKey("message.id"), primary_key=True)
    start_date: Mapped[str] = mapped_column(db.String(20), nullable=True)
    end_date: Mapped[str] = mapped_column(db.String(20), nullable=True)
    duration: Mapped[str] = mapped_column(db.Integer, nullable=True)
    status: Mapped[int] = mapped_column(db.Integer(), nullable=False)

    __mapper_args__ = {
        "polymorphic_identity": "mc_details"
    }
    

    def __init__(self, id, number, body, intent, start_date, end_date, duration, status, timestamp=(datetime.utcnow())):
        super().__init__(id, number, body, intent, timestamp)
        self.start_date = start_date
        self.end_date = end_date
        self.duration = duration
        self.status = status

def create_db():
    with app.app_context():
        db.create_all()
        db.session.commit()


def remove_db():
    with app.app_context():
        db.drop_all()
        db.session.commit()

def seed_db():
    user = User("Rachmiel", "12345678", "rach@rach")
    db.session.add(user)
    db.session.commit()

create_db()




In [5]:
# with app.app_context():
#     seed_db()

In [6]:
conn = sqlite3.connect('chatbot.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM user''')
row = cursor.fetchall()
conn.commit()
conn.close()

print(row)

[]


In [7]:
# hod_df = users[users.index.isin(lookups["hod"])].copy()
# df_replace_spaces(hod_df)

#SECTION start to update df
az_users = users.merge(lookups, how="outer", left_on="name", right_on="name", indicator=True)
az_users = az_users[az_users._merge != "right_only"].drop(columns="_merge")
az_users.sort_values(by="name", inplace=True)

col_order = ['name', 'number', 'email', 'reporting_officer_name', 'hod_name']
az_users = az_users[col_order]

az_users

,name,number,email,reporting_officer_name,hod_name
4,Hariz,34567890,hariz@test.com,Tymothy LimJie,Shawn Tan Minyi
3,Lim Zong Han,88252235,zonghan@go.edu.sg,NaN,NaN
0,Rachmiel Teo Ren Xiang,88584969,rach@go.edu.sg,Lim Zong Han,Lim Zong Han
1,Shawn Tan Minyi,91681054,shawn@go.edu.sg,NaN,NaN
2,Tymothy LimJie,84885787,tymothy@go.edu.sg,Shawn Tan Minyi,Lim Zong Han
5,Vicky,34736482,vicky@grace.com,Rachmiel Teo Ren Xiang,Shawn Tan Minyi


In [8]:
conn = sqlite3.connect('chatbot.db')
cursor = conn.cursor()
cursor.execute('''SELECT * FROM user ORDER BY name ASC''')
db_users = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

conn.commit()
conn.close()

db_users = pd.DataFrame(db_users, columns=column_names)
db_users.sort_index(axis=1, inplace=True)
db_users = db_users[col_order]
db_users

,name,number,email,reporting_officer_name,hod_name


In [9]:
import traceback

exact_match = az_users.equals(db_users)
print(exact_match)

if not exact_match:
    # create 2 dataframes to compare
    
    old_users = pd.merge(az_users, db_users, how="outer", indicator=True).query('_merge == "right_only"').drop(columns='_merge')
    new_users = pd.merge(az_users, db_users, how="outer", indicator=True).query('_merge == "left_only"').drop(columns='_merge')

    update_users = new_users[new_users.name.isin(old_users.name)]

    old_users = old_users[~old_users.name.isin(update_users.name)]
    new_users = new_users[~new_users.name.isin(update_users.name)]

    update_users_tuples = [tuple(update_user) for update_user in update_users.values]
    old_users_tuples = [tuple(old_user) for old_user in old_users.values]
    new_users_tuples = [tuple(new_user) for new_user in new_users.values]
    print(old_users_tuples)
    print(new_users_tuples)


    conn = sqlite3.connect('chatbot.db')
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")

    for name, number, email, reporting_officer, hod in old_users_tuples:
        cursor.execute('DELETE FROM user WHERE name = ?', (name, ))

    for name, number, email, reporting_officer, hod in new_users_tuples:
        cursor.execute('INSERT INTO user (name, number, email) VALUES (?, ?, ?)', (name, number, email))

    conn.commit()
    conn.close()

    conn = sqlite3.connect('chatbot.db')
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
        
    for name, number, email, reporting_officer, hod in update_users_tuples:
        cursor.execute('UPDATE user SET number = ?, email = ?, reporting_officer_name = ?, hod_name = ? WHERE name = ?', (number, email, reporting_officer, hod, name))

    for name, number, email, reporting_officer, hod in new_users_tuples:
        cursor.execute('UPDATE user SET reporting_officer_name = ?, hod_name = ? WHERE name = ?', (reporting_officer, hod, name))

    conn.commit()
    conn.close()
    
    

    
    
   


False
[]
[('Hariz', 34567890, 'hariz@test.com', 'Tymothy LimJie', 'Shawn Tan Minyi'), ('Lim Zong Han', 88252235, 'zonghan@go.edu.sg', nan, nan), ('Rachmiel Teo Ren Xiang', 88584969, 'rach@go.edu.sg', 'Lim Zong Han', 'Lim Zong Han'), ('Shawn Tan Minyi', 91681054, 'shawn@go.edu.sg', nan, nan), ('Tymothy LimJie', 84885787, 'tymothy@go.edu.sg', 'Shawn Tan Minyi', 'Lim Zong Han'), ('Vicky', 34736482, 'vicky@grace.com', 'Rachmiel Teo Ren Xiang', 'Shawn Tan Minyi')]


In [10]:
update_users

,name,number,email,reporting_officer_name,hod_name


In [11]:
old_users

,name,number,email,reporting_officer_name,hod_name


In [12]:
new_users

,name,number,email,reporting_officer_name,hod_name
0,Hariz,34567890,hariz@test.com,Tymothy LimJie,Shawn Tan Minyi
1,Lim Zong Han,88252235,zonghan@go.edu.sg,NaN,NaN
2,Rachmiel Teo Ren Xiang,88584969,rach@go.edu.sg,Lim Zong Han,Lim Zong Han
3,Shawn Tan Minyi,91681054,shawn@go.edu.sg,NaN,NaN
4,Tymothy LimJie,84885787,tymothy@go.edu.sg,Shawn Tan Minyi,Lim Zong Han
5,Vicky,34736482,vicky@grace.com,Rachmiel Teo Ren Xiang,Shawn Tan Minyi


In [13]:
az_users.loc[2, "reporting_officer_name"] = 'Shawn Tan Minyi'

In [14]:
az_users.loc[2, "reporting_officer_name"]

'Shawn Tan Minyi'

In [15]:
with app.app_context():
    first_user = User.query.first()
    hod = first_user.hod


    print(first_user)
    print(first_user.number)
    print(hod)

<User Hariz>
34567890
<User Shawn Tan Minyi>


In [16]:
import uuid

with app.app_context():
    new_mc_details = McDetails(uuid.uuid4().hex, "Rachmiel Teo Ren Xiang", "Hi I would like to take MC for 5 days", "TAKE_MC", '12/10/2023', '14/10/2023', 5, 1)

    db.session.add(new_mc_details)
    db.session.commit()

In [17]:
with app.app_context():
    message = Message.query.first()
    
    print(message.end_date)

14/10/2023


In [26]:
with app.app_context():
    user_me = User.query.filter_by(
        name="Rachmiel Teo Ren Xiang"
    ).all()

ValueError: DataFrame constructor not properly called!